###Set working directory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
%cd /content/drive/My Drive/Face_Recognition_with_SVM/
# !ls

/content/drive/My Drive/Face_Recognition_with_SVM


###Import essential libraries

In [3]:
from PIL import Image
import os
import re

import matplotlib.pyplot as plt
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from skimage.feature import hog

###Obtain image feature & label

In [5]:
folder='a1_TrainingPhotos/' # location of training images
images = []
i=0

X=[]
Y=[]

for filename in os.listdir(folder):   # run the loop for all files in training folder
    img = Image.open(os.path.join(folder,filename)) # open image

    img=img.resize([32,64])  # resize image  to a fixed size

    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=True, channel_axis=-1)

    X.append(fd)

    # obtain training labels from file name

    if re.match("Amir*", filename):
      Yt=0
    elif re.match("Jaya*", filename):
      Yt=1
    elif re.match("Hir*", filename):
      Yt=2
    elif re.match("Anupam*", filename):
      Yt=3
    else:
      Yt=4

    Y.append(Yt)

    i=i+1


In [6]:
fd.shape

(3780,)

In [7]:
target_names=['AamirKhan', 'JayaBadhuri', 'HritikRoshan', 'AnupamKher']

###Split data into training/test

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###Reduce dimenions with PCA

In [9]:
n_components = 25
pca = PCA(n_components=n_components, whiten=True).fit(X_train)

In [10]:
# project training and test data to lower dimension
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [11]:
# Save PCA coefficients for use at prediction

import pickle as pk
pk.dump(pca, open("c1_FR_PCA.pkl","wb"))

###Train SVM classifier

In [12]:
print("Fitting the classifier to the training set")
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(X_train_pca, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.01, probability=True)


In [13]:
# Save best SVM classifier weights for use at prediction

import joblib
joblib.dump(clf.best_estimator_, 'c2_FR_svm_classifier.pkl', compress = 1) # Only best parameters

['c2_FR_svm_classifier.pkl']

###Model performance on test set

In [14]:
y_pred = clf.predict(X_test_pca)
y_pred

array([0, 1, 1, 1, 0, 3, 1, 2, 1, 0, 0, 2, 1, 2, 1, 1, 3, 3, 0, 0, 1, 0,
       0, 0, 0, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 1, 2, 0, 0, 2, 1, 0, 1, 3,
       2, 0, 0, 1, 2, 0, 3, 1, 2, 2, 3, 1, 0, 1, 3, 0, 0, 2, 2, 0, 2, 1,
       0, 0, 3, 1, 3, 1, 1, 0, 1, 2, 1, 1, 2, 0, 2, 1, 2, 0, 3, 2, 1, 0,
       2, 3, 0, 0, 0, 3, 1, 0, 2, 1, 1, 2, 2, 2, 0, 2, 1, 0, 0])

In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.69      0.73        42
           1       0.82      0.69      0.75        39
           2       0.52      0.59      0.55        22
           3       0.25      0.75      0.38         4

    accuracy                           0.67       107
   macro avg       0.59      0.68      0.60       107
weighted avg       0.72      0.67      0.69       107

